In [1]:
import pandas as pd
from datetime import datetime,timedelta
from datetime import date
import numpy as np

In [74]:
# modules for generating the word cloud
from os import path, getcwd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
import wget

In [121]:
import plotly.express as px
import textblob

In [2]:
curdate = date.today()
day_curdate = date.today() -timedelta(days=30)

In [3]:
day_curdate

datetime.date(2020, 12, 27)

In [4]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='96cca94dd1524110b3bf24053f9aebb2')
q = 'corona'
country='us'


In [5]:
iso_code = pd.read_csv("ISO 3166.csv")

In [6]:
iso_code['Alpha-2 code'] = iso_code['Alpha-2 code'].apply(lambda x : str(x).lower())

In [7]:
code_country_name_dict = iso_code.set_index('Alpha-2 code').to_dict()['Country name']

In [8]:
country_name_code_dict = iso_code.set_index('Country name').to_dict()['Alpha-2 code']

In [9]:
code_country_name_dict

{'af': 'Afghanistan',
 'ax': 'Åland Islands',
 'al': 'Albania',
 'dz': 'Algeria',
 'as': 'American Samoa',
 'ad': 'Andorra',
 'ao': 'Angola',
 'ai': 'Anguilla',
 'aq': 'Antarctica',
 'ag': 'Antigua and Barbuda',
 'ar': 'Argentina',
 'am': 'Armenia',
 'aw': 'Aruba',
 'au': 'Australia',
 'at': 'Austria',
 'az': 'Azerbaijan',
 'bs': 'Bahamas (the)',
 'bh': 'Bahrain',
 'bd': 'Bangladesh',
 'bb': 'Barbados',
 'by': 'Belarus',
 'be': 'Belgium',
 'bz': 'Belize',
 'bj': 'Benin',
 'bm': 'Bermuda',
 'bt': 'Bhutan',
 'bo': 'Bolivia (Plurinational State of)',
 'bq': 'Bonaire\r\n\xa0Sint Eustatius\r\n\xa0Saba',
 'ba': 'Bosnia and Herzegovina',
 'bw': 'Botswana',
 'bv': 'Bouvet Island',
 'br': 'Brazil',
 'io': 'British Indian Ocean Territory (the)',
 'bn': 'Brunei Darussalam',
 'bg': 'Bulgaria',
 'bf': 'Burkina Faso',
 'bi': 'Burundi',
 'cv': 'Cabo Verde',
 'kh': 'Cambodia',
 'cm': 'Cameroon',
 'ca': 'Canada',
 'ky': 'Cayman Islands (the)',
 'cf': 'Central African Republic (the)',
 'td': 'Chad',
 'c

In [39]:
country_options = ['ae','ar','at','au','be','bg','br','ca','ch','cn','co','cu','cz','de', 'dk','eg','fr','gb','gr','hk','hu','id','ie','il','in','it','jp','kr','lt','lv','ma','mx','my','ng','nl','no','nz','ph','pl','pt','ro','rs','ru','sa','se','sg','si','sk','th','tr','tw','ua','us','ve','za']
country_options_st = [code_country_name_dict[x] for x in country_options]

In [41]:
iso_code

,Country name,Official state name,Sovereignty,Alpha-2 code,Alpha-3 code,Numeric code,Subdivision code links,Internet ccTLD
0,Afghanistan,The Islamic Republic of Afghanistan,UN member state,af,AFG,4,ISO 3166-2:AF,.af
1,Åland Islands,Åland,Finland,ax,ALA,248,ISO 3166-2:AX,.ax
2,Albania,The Republic of Albania,UN member state,al,ALB,8,ISO 3166-2:AL,.al
3,Algeria,The People's Democratic Republic of Algeria,UN member state,dz,DZA,12,ISO 3166-2:DZ,.dz
4,American Samoa,The Territory of American Samoa,United States,as,ASM,16,ISO 3166-2:AS,.as
...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,The Territory of the Wallis and Futuna Islands,France,wf,WLF,876,ISO 3166-2:WF,.wf
245,Western Sahara,The Sahrawi Arab Democratic Republic,disputed,eh,ESH,732,ISO 3166-2:EH,NaN
246,Yemen,The Republic of Yemen,UN member state,ye,YEM,887,ISO 3166-2:YE,.ye
247,Zambia,The Republic of Zambia,UN member state,zm,ZMB,894,ISO 3166-2:ZM,.zm


# Top News results

In [12]:
# /v2/top-headlines


In [42]:
def create_dataframe_top(queries,country):
    fulldata = pd.DataFrame() 
    for q in queries:
        json_data = newsapi.get_top_headlines(q=q,
                                              language='en',
                                              country=country)
        data = pd.DataFrame(json_data['articles'])
        data['source'] = data['source'].apply(lambda x : x['name'])
        data['publishedAt'] = pd.to_datetime(data['publishedAt'])
        fulldata=pd.concat([fulldata,data])
    fulldata = fulldata.drop_duplicates(subset='url').sort_values(by='publishedAt',ascending=False).reset_index()
    return fulldata

In [43]:
data = create_dataframe_top(['covid','corona'],country='us')

In [44]:
data

,index,source,author,title,description,url,urlToImage,publishedAt,content
0,0,NBC News,Shannon Pettypiece,Biden administration orders additional 200 mil...,The Biden administration is working to purchas...,https://www.nbcnews.com/politics/white-house/b...,https://media1.s-nbcnews.com/j/newscms/2021_04...,2021-01-26 21:30:00+00:00,WASHINGTON The Biden administration is working...
1,1,NPR,,Italy's Prime Minister Quits As COVID-19 Respo...,Giuseppe Conte handed in his resignation Tuesd...,https://www.npr.org/2021/01/26/960744631/italy...,https://media.npr.org/assets/img/2021/01/26/ge...,2021-01-26 18:40:17+00:00,Italian Prime Minister Giuseppe Conte gestures...
2,2,WESH Orlando,None,Central Florida paramedic arrested for stealin...,A paramedic with the Polk County Fire Rescue D...,https://www.wesh.com/article/florida-vaccines-...,https://kubrick.htvapps.com/htv-prod-media.s3....,2021-01-26 17:34:00+00:00,"WINTER HAVEN, Fla. —A paramedic with the Polk ..."
3,3,WBAY,WBAY news staff,Wisconsin to open up COVID-19 vaccinations to ...,"Teachers, child care, and long-term Medicaid r...",https://www.wbay.com/2021/01/26/wisconsin-to-o...,https://gray-wbay-prod.cdn.arcpublishing.com/r...,2021-01-26 17:28:00+00:00,"MADISON, Wis. (WBAY) - More Wisconsin resident..."
4,4,Futurism,None,People Who've Gotten Both Shots Simply Aren't ...,This is really good news.,https://futurism.com/neoscope/israel-covid-vac...,https://wp-assets.futurism.com/2021/01/israel-...,2021-01-26 16:41:10+00:00,Israel has been at the forefront of COVID-19 v...
5,5,TODAY,Kerry Breen,"After getting the COVID-19 vaccine, can I see ...",When can I see friends and family after gettin...,https://www.today.com/health/when-can-families...,https://media1.s-nbcnews.com/j/newscms/2021_04...,2021-01-26 16:27:00+00:00,Many families have spent the last year apart f...
6,6,BBC News,https://www.facebook.com/bbcnews,Blackrock chief: ‘How Covid could help save th...,"Larry Fink, boss of the world's biggest money ...",https://www.bbc.com/news/business-55811332,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-01-26 14:03:00+00:00,Simon JackBusiness editor@BBCSimonJackon Twitt...
7,1,Business Insider,Thomas Colson,The EU is threatening to restrict exports of c...,"""All companies producing vaccines... will have...",https://www.businessinsider.com/vaccine-wars-e...,https://i.insider.com/601003d680e4540019816b85...,2021-01-26 12:03:15+00:00,The EU has threatened to limit exports of coro...


In [45]:
def create_most_recent_markdown(df):
    # img url
    img_path = df['urlToImage'].iloc[0]
    img_alt = df['title'].iloc[0]
    df = df.head(5)
    markdown_str = f"![{img_alt}]({img_path}) <br> "
    for index, row in df.iterrows():
        markdown_str += f"[{row['title']}]({row['url']}) by {row['author']}<br> "
    return markdown_str

In [46]:
top_results_markdown = create_most_recent_markdown(data)

In [47]:
top_results_markdown

"![Biden administration orders additional 200 million doses of Covid-19 vaccine - NBC News](https://media1.s-nbcnews.com/j/newscms/2021_04/3445192/210126-pfizer-vaccine-doses-jm-1615_bbf4815b1a28acba8f09596875b7fb75.nbcnews-fp-1200-630.jpg) <br> [Biden administration orders additional 200 million doses of Covid-19 vaccine - NBC News](https://www.nbcnews.com/politics/white-house/biden-administration-orders-additional-200-million-doses-covid-19-vaccine-n1255744) by Shannon Pettypiece<br> [Italy's Prime Minister Quits As COVID-19 Response Splits Coalition Government - NPR](https://www.npr.org/2021/01/26/960744631/italys-prime-minister-quits-as-covid-19-response-splits-coalition-government) by <br> [Central Florida paramedic arrested for stealing vaccines, officials say - WESH 2 Orlando](https://www.wesh.com/article/florida-vaccines-stolen/35321984) by None<br> [Wisconsin to open up COVID-19 vaccinations to more groups starting March 1 - WBAY](https://www.wbay.com/2021/01/26/wisconsin-to-o

![MLB players open to games without fans amid coronavirus uncertainty, union chief says - Yahoo Sports](https://s.yimg.com/uu/api/res/1.2/2ycLYacf7NbGCyddmUEYFQ--~B/aD0zMjU3O3c9NTA4ODtzbT0xO2FwcGlkPXl0YWNoeW9u/https://media-mbst-pub-ue1.s3.amazonaws.com/creatr-images/2020-03/19ffe030-709c-11ea-bbe3-9281d529cfc9) <br> [MLB players open to games without fans amid coronavirus uncertainty, union chief says - Yahoo Sports](https://sports.yahoo.com/mlb-players-open-to-games-without-fans-amid-coronavirus-uncertainty-union-chief-says-024051838.html) by Tim Brown<br> [Cuomo calls Trump 'incorrect and grossly uninformed' about New York's ventilator situation as fight escalates - CNN](https://www.cnn.com/2020/03/27/politics/cuomo-trump-incorrect-and-grossly-uninformed/index.html) by Shimon Prokupecz and Caroline Kelly, CNN<br> [Google has banned the Infowars Android app over false coronavirus claims - The Verge](https://www.theverge.com/2020/3/27/21197701/google-banned-infowars-android-app-play-store-coronavirus) by Jay Peters<br> [The mystery behind Germany's low coronavirus death rate - CBS News](https://www.cbsnews.com/news/coronavirus-death-rate-mystery-germany-italy-covid-19/) by CBS News<br> [Teen who may have died of coronavirus was turned away from urgent care due to lack of insurance | TheHill - The Hill](https://thehill.com/policy/healthcare/489957-teen-who-may-have-died-of-coronavirus-was-turned-away-from-urgent-care-due) by Brooke Seipel<br>

# Create Dataframe For the Last 30 days

In [107]:
def get_sources(country):
    sources = newsapi.get_sources(country=country)
    sources = [x['id'] for x in sources['sources']]
    return sources

In [108]:
def create_dataframe_last_30d(queries, sources):
    fulldata = pd.DataFrame()
    for q in queries:
        for s in sources:
            print(s)
            json_data = newsapi.get_everything(q=q,
                                              language='en',
                                                from_param=str(date.today() -timedelta(days=29)),
                                              to= str(date.today()),
                                               sources = s,
                                              page_size=100,
                                              page = 1,sort_by='relevancy'
                                              )
            if len(json_data['articles'])>0:
                data = pd.DataFrame(json_data['articles'])
                fulldata=pd.concat([fulldata,data])
    if len(fulldata)>0:
        fulldata['source'] = fulldata['source'].apply(lambda x : x['name'])
        fulldata['publishedAt'] = pd.to_datetime(fulldata['publishedAt'])
        fulldata = fulldata.drop_duplicates(subset='url').sort_values(by='publishedAt',ascending=False).reset_index()
    return fulldata
    

In [109]:
sources = get_sources(country='us')

In [110]:
fulldf = create_dataframe_last_30d(['corona'],sources)

abc-news
al-jazeera-english
ars-technica
associated-press
axios
bleacher-report
bloomberg
breitbart-news
business-insider
buzzfeed
cbs-news
cnn
cnn-es
crypto-coins-news
engadget
entertainment-weekly
espn
espn-cric-info
fortune
fox-news
fox-sports
google-news
hacker-news
ign


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [111]:
fulldf.head()

,index,source,author,title,description,url,urlToImage,publishedAt,content,story_sentiment
0,3,Vice News,"Jenna Mahale, Ryan White",Watch the creepy new trailer for Netflix’s tru...,Time to check into the infamous hotel that hou...,https://i-d.vice.com/en_uk/article/jgqkxb/watc...,https://video-images.vice.com/articles/601059b...,2021-01-26 18:31:15+00:00,Known as a place where serial killers let thei...,-0.025758
1,21,Reuters,Breakingviews columnists,"Breakingviews - Corona Capital: IMF forecasts,...",Corona Capital is a column updated throughout ...,https://www.reuters.com/article/us-health-coro...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-26 17:00:00+00:00,NEW YORK/MILAN/LONDON/HONG KONG (Reuters Break...,-0.041667
2,0,Breitbart News,Oliver JJ Lane,"Dutch Coronavirus Riots, Worst in 40 Years, De...",The Dutch government has denied it was plannin...,https://www.breitbart.com/europe/2021/01/26/du...,https://media.breitbart.com/media/2021/01/Gett...,2021-01-26 16:00:51+00:00,The Dutch government denied it was planning to...,-1.000000
3,38,Reuters,Reuters Staff,Turkey bans parties at ski resorts after COVID...,Turkey has banned music and parties at ski res...,https://www.reuters.com/article/health-coronav...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-26 14:56:00+00:00,By Reuters Staff\r\nANKARA (Reuters) - Turkey ...,-0.700000
4,3,Newsweek,Soo Kim,Gavin Newsom Removes Essential Workers From Va...,The announcement comes as many service industr...,https://www.newsweek.com/coronavirus-californi...,https://d.newsweek.com/en/full/1716311/califor...,2021-01-26 12:44:55+00:00,California's regional stay-at-home order was l...,0.166667


In [113]:
def get_data(date):
    url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
    filename = wget.download(url,"cases.csv")
    casedata = pd.read_csv(filename, encoding='latin-1')
    casedata['dateRep']=pd.to_datetime(casedata['dateRep'],format = "%d/%m/%Y")
    mindt = min(casedata['dateRep'])
    casedata['days_since_start'] = casedata['dateRep'].apply(lambda x: (x -  mindt).days)
    #casedata['first_case_date'] 
    country_first_case_date = casedata[casedata['cases_weekly']!=0].groupby("countryterritoryCode").aggregate({'dateRep':'min'}).reset_index()
    country_first_case_date.columns = ['countryterritoryCode','firstcasedate']
    casedata = casedata.merge(country_first_case_date,on='countryterritoryCode',how='left')
    casedata['days_since_country_first_case'] = casedata.apply(lambda x: (x['dateRep'] - x['firstcasedate']).days,axis=1)
    casedata = casedata[casedata['days_since_country_first_case']>=0]
    casedata['geoId'] = casedata['geoId'].apply(lambda x:str(x).lower())
    casedata = casedata.sort_values(by='dateRep')
    casedata['cumdeaths']= casedata.groupby(by=['countryterritoryCode'])['deaths_weekly'].apply(lambda x: x.cumsum())
    casedata['cumcases']= casedata.groupby(by=['countryterritoryCode'])['cases_weekly'].apply(lambda x: x.cumsum())
    return casedata

In [114]:
url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
filename = wget.download(url,"cases.csv")
casedata = pd.read_csv(filename, encoding='latin-1')
casedata['dateRep']=pd.to_datetime(casedata['dateRep'],format = "%d/%m/%Y")
mindt = min(casedata['dateRep'])
casedata['days_since_start'] = casedata['dateRep'].apply(lambda x: (x -  mindt).days)

In [115]:
casedata

,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days,days_since_start
0,2021-01-18,2021-02,557,45,Afghanistan,AF,AFG,38041757.0,Asia,3.24,378
1,2021-01-11,2021-01,675,71,Afghanistan,AF,AFG,38041757.0,Asia,4.15,371
2,2021-01-04,2020-53,902,60,Afghanistan,AF,AFG,38041757.0,Asia,7.61,364
3,2020-12-28,2020-52,1994,88,Afghanistan,AF,AFG,38041757.0,Asia,7.19,357
4,2020-12-21,2020-51,740,111,Afghanistan,AF,AFG,38041757.0,Asia,6.56,350
...,...,...,...,...,...,...,...,...,...,...,...
10000,2020-04-20,2020-16,11,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.11,105
10001,2020-04-13,2020-15,5,2,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05,98
10002,2020-04-06,2020-14,2,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05,91
10003,2020-03-30,2020-13,5,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05,84


In [116]:
casedata=get_data(date)

In [117]:
#case growth curve for multiple countries
casedata[casedata['geoId']=='dk']

,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days,days_since_start,firstcasedate,days_since_country_first_case,cumdeaths,cumcases
2560,2020-03-02,2020-09,4,0,Denmark,dk,DNK,5806081.0,Europe,0.07,56,2020-03-02,0.0,0,4
2559,2020-03-09,2020-10,34,0,Denmark,dk,DNK,5806081.0,Europe,0.65,63,2020-03-02,7.0,0,38
2558,2020-03-16,2020-11,837,1,Denmark,dk,DNK,5806081.0,Europe,15.00,70,2020-03-02,14.0,1,875
2557,2020-03-23,2020-12,520,12,Denmark,dk,DNK,5806081.0,Europe,23.37,77,2020-03-02,21.0,13,1395
2556,2020-03-30,2020-13,1000,59,Denmark,dk,DNK,5806081.0,Europe,26.18,84,2020-03-02,28.0,72,2395
2555,2020-04-06,2020-14,1974,107,Denmark,dk,DNK,5806081.0,Europe,51.22,91,2020-03-02,35.0,179,4369
2554,2020-04-13,2020-15,1805,94,Denmark,dk,DNK,5806081.0,Europe,65.09,98,2020-03-02,42.0,273,6174
2553,2020-04-20,2020-16,1210,82,Denmark,dk,DNK,5806081.0,Europe,51.93,105,2020-03-02,49.0,355,7384
2552,2020-04-27,2020-17,1191,67,Denmark,dk,DNK,5806081.0,Europe,41.35,112,2020-03-02,56.0,422,8575
2551,2020-05-04,2020-18,948,59,Denmark,dk,DNK,5806081.0,Europe,36.84,119,2020-03-02,63.0,481,9523


In [118]:
px.line(casedata,x='days_since_country_first_case',y='cumcases',line_group='countriesAndTerritories',color = 'countriesAndTerritories')

In [119]:
px.line(casedata,x='days_since_country_first_case',y='cumdeaths',line_group='countryterritoryCode',color = 'countryterritoryCode')

In [120]:
fig = px.choropleth(casedata, locations="countryterritoryCode", color="cumdeaths", hover_name="countriesAndTerritories", animation_frame="days_since_start",range_color=[0,max(casedata.cumdeaths)])
fig.show()